# Let's try to use python with sqlite

In [1]:
import sqlite3

In [2]:
# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

In [3]:
# Create tables for authors, genres, and books

cursor.execute('''
    CREATE TABLE IF NOT EXISTS authors (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        birth_year INTEGER
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS genres (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        genre_name TEXT NOT NULL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS books (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        author_id INTEGER,
        genre_id INTEGER,
        publication_year INTEGER,
        pages INTEGER,
        FOREIGN KEY (author_id) REFERENCES authors(id),
        FOREIGN KEY (genre_id) REFERENCES genres(id)
    )
''')

# Commit the changes
conn.commit()

In [4]:
# Function to insert a new author
def add_author(name, birth_year):
    cursor.execute('''
        INSERT INTO authors (name, birth_year)
        VALUES (?, ?)
    ''', (name, birth_year))
    conn.commit()

# Function to insert a new genre
def add_genre(genre_name):
    cursor.execute('''
        INSERT INTO genres (genre_name)
        VALUES (?)
    ''', (genre_name,))
    conn.commit()

# Function to insert a new book
def add_book(title, author_id, genre_id, publication_year, pages):
    cursor.execute('''
        INSERT INTO books (title, author_id, genre_id, publication_year, pages)
        VALUES (?, ?, ?, ?, ?)
    ''', (title, author_id, genre_id, publication_year, pages))
    conn.commit()

In [5]:
# Add authors
add_author('J.K. Rowling', 1965)
add_author('George Orwell', 1903)

# Add genres
add_genre('Fantasy')
add_genre('Dystopian')

# Get the author and genre IDs
cursor.execute('SELECT id FROM authors WHERE name="J.K. Rowling"')
author_id = cursor.fetchone()[0]

cursor.execute('SELECT id FROM genres WHERE genre_name="Fantasy"')
genre_id = cursor.fetchone()[0]

# Add books
add_book('Harry Potter and the Sorcerer\'s Stone', author_id, genre_id, 1997, 309)

cursor.execute('SELECT id FROM authors WHERE name="George Orwell"')
author_id = cursor.fetchone()[0]

cursor.execute('SELECT id FROM genres WHERE genre_name="Dystopian"')
genre_id = cursor.fetchone()[0]

add_book('1984', author_id, genre_id, 1949, 328)

# Commit changes
conn.commit()

In [6]:
# Query all books and their authors
def get_books():
    cursor.execute('''
        SELECT books.title, authors.name, genres.genre_name, books.publication_year
        FROM books
        JOIN authors ON books.author_id = authors.id
        JOIN genres ON books.genre_id = genres.id
    ''')
    return cursor.fetchall()

# Example: Get all books
books = get_books()
for book in books:
    print(f'Title: {book[0]}, Author: {book[1]}, Genre: {book[2]}, Year: {book[3]}')

# Query books by a specific genre
def get_books_by_genre(genre_name):
    cursor.execute('''
        SELECT books.title, authors.name, genres.genre_name, books.publication_year
        FROM books
        JOIN authors ON books.author_id = authors.id
        JOIN genres ON books.genre_id = genres.id
        WHERE genres.genre_name = ?
    ''', (genre_name,))
    return cursor.fetchall()

# Example: Get books in the "Fantasy" genre
fantasy_books = get_books_by_genre('Fantasy')
for book in fantasy_books:
    print(f'Title: {book[0]}, Author: {book[1]}, Genre: {book[2]}, Year: {book[3]}')

# Query books by publication year
def get_books_by_year(start_year, end_year):
    cursor.execute('''
        SELECT books.title, authors.name, genres.genre_name, books.publication_year
        FROM books
        JOIN authors ON books.author_id = authors.id
        JOIN genres ON books.genre_id = genres.id
        WHERE books.publication_year BETWEEN ? AND ?
    ''', (start_year, end_year))
    return cursor.fetchall()

# Example: Get books published between 1900 and 2000
books_1900_2000 = get_books_by_year(1900, 2000)
for book in books_1900_2000:
    print(f'Title: {book[0]}, Author: {book[1]}, Genre: {book[2]}, Year: {book[3]}')

Title: Harry Potter and the Sorcerer's Stone, Author: J.K. Rowling, Genre: Fantasy, Year: 1997
Title: 1984, Author: George Orwell, Genre: Dystopian, Year: 1949
Title: Harry Potter and the Sorcerer's Stone, Author: J.K. Rowling, Genre: Fantasy, Year: 1997
Title: Harry Potter and the Sorcerer's Stone, Author: J.K. Rowling, Genre: Fantasy, Year: 1997
Title: 1984, Author: George Orwell, Genre: Dystopian, Year: 1949


In [7]:
conn.close()